# Analyzing your Runs

In this tutorial you will learn: 

1. How to visualize and analyze your runs

In [5]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
from bokeh.palettes import Colorblind
output_notebook()
import numpy as np
import pandas as pd
import astropy.units as u

#here is pyeddy
import virga.justdoit as jdi
import virga.justplotit as jpi

Loading BokehJS ...

This time let's load in a Hot Jupiter PT profile and go through the motions of the analysis. 

In [8]:
mieff_dir = '/Users/tkataria/Research/virga/optics/'

metallicity = 1 #atmospheric metallicity relative to Solar
mean_molecular_weight = 2.2 # atmospheric mean molecular weight

#set the run 
a = jdi.Atmosphere(['CaTiO3','CaAl12O19'],fsed=1,mh=metallicity, mmw = mean_molecular_weight)

#set the planet gravity
a.gravity(gravity=7.460, gravity_unit=u.Unit('m/(s**2)'))

#Get preset pt profile for testing
a.ptk(df = jdi.hot_jupiter())

#get full dictionary output 
all_out = jdi.compute(a, as_dict=True,directory=mieff_dir)

Virtual Cloud Found: CaAl12O19


## Virtual Cloud 

The first thing that has come up in this run is a new warning message regarding the _virtual cloud_. 

This message indicates that one or more of the condensibles that was entered (in our case, MgSiO3 and Fe) is likely to condense below the bounds of the grid that was supplied. 

**What to do**:
1. Either remove the gas, if it condensing WELL below 
2. Supply a grid that goes down to lower pressures. 

## PT Profile with Condensation Curves

Thick lines in this figure signify the gases that were supplied in the run, NOT those that were recommended by `pyeddy.recommend`. 

In [9]:
show(jpi.pt(all_out,plot_height=450))

## Cumulative Optical Depth By Gas 

This is the cumulative optical depth for the top of the atmosphere down. Remember that this is the optical depth for conservative geometric scatterers. They won't identically match up with the `opd_per_layer` output shown below. This quantity, therefore, is just to help you visualize about where and how optically thick the cloud is. The actual quantities in `opd_per_layer` should be used for any real calculations

In [10]:
show(jpi.opd_by_gas(all_out))

## Visualize Optical Depth, Asymmetry, and Single Scattering

The key to analyzing these plots: 

1. Start with the middle plot, locate the cloud decks
2. Scan over to the single scattering and asymmetry plots and get a sense for what the optical properties are doing where the cloud decks are

In [11]:
show(jpi.all_optics(all_out))


## Particle Radii 

Currently, all particle distributions in `pyeddy` are log-normal. There is an input (`sig`) in the run call if you'd like to alter the width.

This figure is important for two things: 
1. Determining the full profile of particle radii in your atmosphere 
2. Assessing whether or not the grid that was used to compute Mie properties was sufficient. **If your distribution is cut off at the large or small end, consider changing `rmin` and `nradii` in `pyeddy.calc_mie_db`.** 

In [12]:
fig, dndr = jpi.radii(all_out,at_pressure=1e-3)
show(fig)


## Mean Mass Mixing Ratio of the Condensates

In [13]:
show(jpi.condensate_mmr(all_out))